In [1]:
import pandas as pd
#1. Load the data
players_file = "/Users/noahpockriss/Winter Sprint 2026/data/players.csv"
players = pd.read_csv(players_file)
appearances_file = "/Users/noahpockriss/Winter Sprint 2026/data/appearances.csv"
appearances = pd.read_csv(appearances_file)
valuations_file = "/Users/noahpockriss/Winter Sprint 2026/data/player_valuations.csv"
valuations = pd.read_csv(valuations_file)

#2. Inspect the columns (Look for the "key" to join them together)
print ("Players Columns:", players.columns.tolist())
print("-" * 20)
print ("Appearances Columns:", appearances.columns.tolist())


Players Columns: ['player_id', 'first_name', 'last_name', 'name', 'last_season', 'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth', 'country_of_citizenship', 'date_of_birth', 'sub_position', 'position', 'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name', 'image_url', 'url', 'current_club_domestic_competition_id', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']
--------------------
Appearances Columns: ['appearance_id', 'game_id', 'player_id', 'player_club_id', 'player_current_club_id', 'date', 'player_name', 'competition_id', 'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played']


In [2]:
#1. Filter for the 2024/2025 season
season_year = 2025
# convert date to datetime (in-place, safe to reuse variable)
appearances['date'] = pd.to_datetime(appearances['date'], errors='coerce')
# assume season runs from August 1 of previous year to May 30 of season_year
season_start = pd.Timestamp(year=season_year - 1, month=8, day=1)
season_end = pd.Timestamp(year=season_year, month=5, day=30)
appearances_recent = appearances[(appearances['date'] >= season_start) & (appearances['date'] <= season_end)]

#2. Group by player_id and sum their stats
player_stats = appearances_recent.groupby('player_id').agg({
    'goals': 'sum',
    'assists': 'sum',
    'minutes_played': 'sum',
    'yellow_cards': 'sum',
    'red_cards': 'sum',
    'game_id': 'count'  # Number of games played
}).reset_index()
#Rename game_id to games_played
player_stats.rename(columns={'game_id': 'games_played'}, inplace=True)

#Check the result
print(player_stats.head())
    
    

   player_id  goals  assists  minutes_played  yellow_cards  red_cards  \
0       3333      0        0             172             1          0   
1       5336      0        0             923             3          0   
2       7161      2        2             545             0          0   
3       7825      0        0             990             0          0   
4      11530      0        0            1798             2          0   

   games_played  
0             3  
1            15  
2            13  
3            11  
4            23  


In [3]:
# Merge Players + Stats
df = pd.merge(players, player_stats, on='player_id', how='inner')

#Peek at the result
print(f"We have {len(df)} players with stats for the {season_year-1}, {season_year} season.")
print(df[['name', 'goals', 'assists', 'minutes_played','market_value_in_eur']].head())


We have 5834 players with stats for the 2024, 2025 season.
                  name  goals  assists  minutes_played  market_value_in_eur
0         James Milner      0        0             172            1000000.0
1  Anastasios Tsokanis      0        0             923             300000.0
2        Jonas Hofmann      2        2             545            3000000.0
3           Pepe Reina      0        0             990             600000.0
4        Lionel Carole      0        0            1798             400000.0


In [4]:
#1. Convert date_of_birth to actual datetime objects
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'])

#2. Calculate age as of a specific date (e.g., Jan 5, 2026)
# Subtract the DOB from today and divide by 365.25 to get age in years
now = pd.Timestamp.now()
df['age'] = (now - df['date_of_birth']).dt.days / 365.25

#3. Handle missing data
# If the player has no birthday fill with the average age
df['age'] = df['age'].fillna(df['age'].median())

#4. Make it clean integers
df['age'] = df['age'].astype(int)

#Check the first 5 rows
print(df[['name', 'date_of_birth', 'age']].head())

                  name date_of_birth  age
0         James Milner    1986-01-04   40
1  Anastasios Tsokanis    1991-05-02   34
2        Jonas Hofmann    1992-07-14   33
3           Pepe Reina    1982-08-31   43
4        Lionel Carole    1991-04-12   34


In [5]:
#1. Drop Goalkeepers (their transfer logic is totally different)
# 'sub_position' column indicates player position
df = df[df['sub_position'] != 'Goalkeeper']

#2. Drop players with 0 market value (Missing data)
df = df[df['market_value_in_eur'] > 0]

#3. Drop players who barely played (e.g. less than 300 minutes total)
df = df[df['minutes_played'] >= 300]

print(f"Cleaned dataset has {len(df)} players ready for analysis.")

#Save the cleaned data to a new CSV
df.to_csv('/Users/noahpockriss/Winter Sprint 2026/data/final_data.csv', index=False)

Cleaned dataset has 4531 players ready for analysis.
